# Environment setup

In [ ]:
%pip install --upgrade --quiet langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.5/411.5 kB 16.1 MB/s eta 0:00:00


In [ ]:
%pip install -qU langchain-openai
%pip install --upgrade "httpx<0.28"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

# Prompting without Message objects

In [ ]:
response=model.invoke("Give me a list of 4 tourist locations in Chicago")
response.content

'Certainly! Here are four popular tourist locations in Chicago:\n\n1. **Millennium Park**: Known for its iconic Cloud Gate sculpture (often referred to as "The Bean"), Millennium Park is a must-visit for its beautiful gardens, art installations, and the Jay Pritzker Pavilion.\n\n2. **The Art Institute of Chicago**: This world-renowned art museum houses an extensive collection of artworks, including pieces by famous artists like Grant Wood, Georges Seurat, and Vincent van Gogh.\n\n3. **Navy Pier**: A popular entertainment destination, Navy Pier offers rides, restaurants, shops, and attractions such as the Centennial Wheel, a large Ferris wheel offering stunning views of the city skyline and Lake Michigan.\n\n4. **The Field Museum**: One of the largest natural history museums in the world, The Field Museum is home to a vast range of exhibits, including the famous T. rex skeleton named Sue.\n\nThese locations offer a mix of art, culture, and entertainment, making them ideal for visitors t

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate(template="Give me a list of {number} tourist locations in {city}")

prompt = prompt_template.invoke({"city": "Chicago", "number": 4})
response = model.invoke(prompt)

response.content

'Certainly! Here are four popular tourist locations in Chicago:\n\n1. **Millennium Park**: Known for its iconic Cloud Gate sculpture, often referred to as "The Bean," Millennium Park is a must-visit for its beautiful gardens, art installations, and outdoor concerts.\n\n2. **The Art Institute of Chicago**: This world-renowned museum boasts an impressive collection of art from around the globe, including famous works like Grant Wood\'s "American Gothic" and Georges Seurat\'s "A Sunday on La Grande Jatte."\n\n3. **Navy Pier**: A lively destination along Lake Michigan, Navy Pier offers entertainment options, dining, boat tours, and attractions like the Centennial Wheel, providing stunning views of the city skyline.\n\n4. **Willis Tower Skydeck**: Formerly known as the Sears Tower, Willis Tower features the Skydeck on the 103rd floor, where visitors can step out onto "The Ledge," a glass balcony extending outside the building, offering breathtaking views of Chicago.\n\nThese locations provi

# Structured output

In [ ]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate

class Location(BaseModel):
    name: str = Field(description="Name of the location to visit")
    description: str = Field(description="Brief description of the location")
    time_to_be_spent: str = Field(description="Time I should spent")

class LocationsList(BaseModel):
    locations: list[Location] = Field(description="List of locations")

# LLM
model = ChatOpenAI(model="gpt-4o").with_structured_output(
    LocationsList
)


prompt_template=PromptTemplate(template="Give me {number} tourist locations in {city}")
prompt=prompt_template.invoke({"city": "Chicago", "number": 7})

response=model.invoke(prompt)
response.model_dump()["locations"]

[{'name': 'Millennium Park',
  'description': 'A famous public park known for its art installations, gardens, and the iconic Cloud Gate sculpture.',
  'time_to_be_spent': '2-3 hours'},
 {'name': 'The Art Institute of Chicago',
  'description': 'One of the oldest and largest art museums in the United States, featuring an extensive collection of artworks.',
  'time_to_be_spent': '3-4 hours'},
 {'name': 'Navy Pier',
  'description': 'A popular destination along Lake Michigan offering entertainment, dining, and attractions such as the Centennial Wheel.',
  'time_to_be_spent': '3-4 hours'},
 {'name': 'Willis Tower Skydeck',
  'description': 'An observation deck located on the 103rd floor of the Willis Tower, offering panoramic views of Chicago.',
  'time_to_be_spent': '1-2 hours'},
 {'name': 'Shedd Aquarium',
  'description': 'A large indoor public aquarium with a diverse range of marine life exhibits.',
  'time_to_be_spent': '2-3 hours'},
 {'name': 'Field Museum',
  'description': 'A natur

# Structured output with LCEL

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class Locations(BaseModel):
    name: str = Field(description="Name of the location to visit")
    description: str = Field(description="Brief description of the location")
    time_to_be_spent: str = Field(description="Time I should spent")

class LocationsList(BaseModel):
    locations: list[Locations] = Field(description="List of locations")

# LLM
model = ChatOpenAI(model="gpt-4o")

parser = JsonOutputParser(pydantic_object=LocationsList)

prompt_template = PromptTemplate(template="Give me a list of {number} tourist locations in {city}\n{format_instructions}",
                                 input_variables=["number","city"],
                                 partial_variables={"format_instructions": parser.get_format_instructions()})

chain = prompt_template | model | parser


response = chain.invoke({"number": 7, "city": "Chicago"})

response

{'locations': [{'name': 'Millennium Park',
   'description': 'A famous civic center and art park known for the Cloud Gate sculpture.',
   'time_to_be_spent': '2 hours'},
  {'name': 'The Art Institute of Chicago',
   'description': 'One of the oldest and largest art museums in the United States.',
   'time_to_be_spent': '3 hours'},
  {'name': 'Navy Pier',
   'description': 'A popular tourist destination featuring shops, eateries, and a Ferris wheel.',
   'time_to_be_spent': '2 hours'},
  {'name': 'Willis Tower Skydeck',
   'description': 'Offers a panoramic view of the city from the top of one of the tallest buildings in the world.',
   'time_to_be_spent': '1 hour'},
  {'name': 'The Field Museum',
   'description': 'A natural history museum with extensive exhibits on fossils, cultures, and more.',
   'time_to_be_spent': '3 hours'},
  {'name': 'Shedd Aquarium',
   'description': 'A large indoor public aquarium with a wide variety of aquatic animals.',
   'time_to_be_spent': '2 hours'},
 

In [ ]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Locations": {"properties": {"name": {"description": "Name of the location to visit", "title": "Name", "type": "string"}, "description": {"description": "Brief description of the location", "title": "Description", "type": "string"}, "time_to_be_spent": {"description": "Time I should spent", "title": "Time To Be Spent", "type": "string"}}, "required": ["name", "description", "time_to_be_spent"], "title": "Locations", "type": "object"}}, "properties": {"locations": {"description": "List of locations", "items": {"$ref": "#/$def

# Message objects, Output parser and LCEL

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class Locations(BaseModel):
    name: str = Field(description="Name of the location to visit")
    description: str = Field(description="Brief description of the location")
    time_to_be_spent: str = Field(description="Time I should spent")

class LocationsList(BaseModel):
    locations: list[Locations] = Field(description="List of locations")

# LLM
model = ChatOpenAI(model="gpt-4o")

In [ ]:
system_template = """You are a travel agent. You operate in {region} only.
    Help in best possible way, but say sorry and no to anyone who asks for help outside the area you operate. \n{format_instructions}"""
user_template = "Plan a {days} day trip to {city}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", user_template)],
)

parser = JsonOutputParser(pydantic_object=LocationsList)
chain = prompt_template | model | parser


response = chain.invoke({"region": "Middle East", "days": 5, "city": "Dubai", "format_instructions": parser.get_format_instructions()})

response["locations"]

[{'name': 'Burj Khalifa',
  'description': "Visit the world's tallest building and enjoy panoramic views of the city.",
  'time_to_be_spent': '2-3 hours'},
 {'name': 'Dubai Mall',
  'description': "Explore one of the world's largest shopping malls with an aquarium and ice rink.",
  'time_to_be_spent': '4-5 hours'},
 {'name': 'Dubai Marina',
  'description': 'Enjoy a walk along the Marina, dine at restaurants, and take a boat tour.',
  'time_to_be_spent': '3-4 hours'},
 {'name': 'Jumeirah Beach',
  'description': 'Relax on the white sands and enjoy the view of the Burj Al Arab.',
  'time_to_be_spent': '3-4 hours'},
 {'name': 'Desert Safari',
  'description': 'Experience dune bashing, camel riding, and a traditional Bedouin camp dinner.',
  'time_to_be_spent': '6-7 hours'},
 {'name': 'Dubai Creek',
  'description': 'Take an abra ride along the creek and explore the traditional souks.',
  'time_to_be_spent': '3-4 hours'},
 {'name': 'The Dubai Fountain',
  'description': 'Watch the spectac

# Classification problem

## Sentiment analysis

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser


# LLM
model = ChatOpenAI(temperature=0, model="gpt-4o")

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}\n{format_instructions}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text", enum=["happy", "negative", "neutral"])
    aggressiveness: int = Field(
        description="How aggressive the text is? higher the number, more aggressive it is."
        , enum=[1,2,3,4,5]
    )
    language: str = Field(description="The language the text is written in", enum=["Hindi", "English", "French", "Unknown"])

parser = JsonOutputParser(pydantic_object=Classification)

chain = tagging_prompt | model | parser

In [ ]:
response = chain.invoke({"input": "你是個白痴。我不喜歡你。", "format_instructions": parser.get_format_instructions()})
response

{'sentiment': 'negative', 'aggressiveness': 4, 'language': 'Unknown'}